In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re

In [2]:
# Формирование исходных URL
base_url = "http://books.toscrape.com/"
url = base_url + "catalogue/page-1.html"

In [3]:
pages_counter = 0 ### для подсчета количества обработанных веб-страниц
books = [] ### для сохранения распарсенных книг 

In [7]:
while url:
    print("\nCкрапинг страницы №", pages_counter+1)
    # Отправка GET запроса по URL
    response = requests.get(url)

    # Парсинг HTML страницы с использованием BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")

    h3_tags = soup.find_all("h3")

    for h3_tag in h3_tags:

        # Поиск тега <a> 
        a_tag = h3_tag.find("a", href=True)

        # Создание ссылки на страницу книги
        book_url = base_url + "catalogue/" + a_tag["href"]

        # Отправка запроса по ссылке на книгу
        book_response = requests.get(book_url)

        # Парсинг страницы книги
        book_soup = BeautifulSoup(book_response.text, "html.parser")

        ### экстракция требуемых данных о книге
        title = book_soup.find("h1").text.strip()
        price = float(book_soup.find("p", class_="price_color").text.strip().replace("Â\u00a3", ""))
        stock = int(re.findall(r'\d+', book_soup.find("p", class_="instock availability").text.strip())[0])
        description = book_soup.find("meta", attrs={"name": "description"})["content"]

        # Добавление информации о книге в общий список
        books.append({
                "Название": title,
                "Цена в фунтах стерлингов": price,
                "Количество в наличии": stock,
                "Описание": description
            })
        print("Добавили книгу", title)
    
    next_button = soup.find('a', string='next')
        
    ### Проверка, есть ли ссылка "next" на следующую страницу
    if next_button:
        url = base_url + "catalogue/" + next_button['href']
        pages_counter += 1
    else:
        url = None  # если ссылки нет - выходим из цикла while


Cкрапинг страницы № 48
Добавили книгу The Bhagavad Gita
Добавили книгу The Bette Davis Club
Добавили книгу The Art of Not Breathing
Добавили книгу Taking Shots (Assassins #1)
Добавили книгу Starlark
Добавили книгу Skip Beat!, Vol. 01 (Skip Beat! #1)
Добавили книгу Sister Sable (The Mad Queen #1)
Добавили книгу Shatter Me (Shatter Me #1)
Добавили книгу Shameless
Добавили книгу Shadow Rites (Jane Yellowrock #10)
Добавили книгу Settling the Score (The Summer Games #1)
Добавили книгу Sense and Sensibility
Добавили книгу Saga, Volume 1 (Saga (Collected Editions) #1)
Добавили книгу Rhythm, Chord & Malykhin
Добавили книгу Rat Queens, Vol. 1: Sass & Sorcery (Rat Queens (Collected Editions) #1-5)
Добавили книгу Paradise Lost (Paradise #1)
Добавили книгу Paper Girls, Vol. 1 (Paper Girls #1-5)
Добавили книгу Ouran High School Host Club, Vol. 1 (Ouran High School Host Club #1)
Добавили книгу Origins (Alphas 0.5)
Добавили книгу One Second (Seven #7)

Cкрапинг страницы № 49
Добавили книгу On the Ro

In [8]:
print("\nСуммарное число книг: ", len(books)) 
### результат расчета суммарного числа распарсенных книг соответствует указанному на сайте


Суммарное число книг:  1033


In [9]:
### сохранение общего списка в файл кодировкой, содержащей буквы русского алфавита
with open("books_from_books.toscrape.com.json", "w", encoding='utf-8') as f:
        json.dump(books, f, indent=4, ensure_ascii=False)